<a href="https://colab.research.google.com/github/datjandra/GPDoc/blob/main/gpdoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTRUCTIONS:

1. Get a HuggingFace token from huggingface.co. Copy the token, find the section below that says HF_TOKEN=HUGGINGFACE_TOKEN, and paste the token there. Run the cell and verify the token is printed in output.
2. In left navbar click Files icon. Right click anywhere in the Files section and create a new folder called patient_data.
3. Add doctor's notes in .txt file format to patient_data folder. Below is an example note. **Don't** use notes with real patient data!  
4. Run the code below then it will try to find answers from the notes. Change the questions in the code if you like and rerun.
5. If you find issues like missing imports then try restarting session under Runtime menu.


```
Personal Data and Overall Health
September 1, 2006

Problems
Asthma, unspecified without mention of status asthmaticus

Medications
Flovent 110 mcg/inh aerosol with adapter Glaxo Wellcome
Singulair 5mg tablet chewable Merck & Co. Inc
Albuterol Sulfate 4mg tablet ESI Lederle Generics
Sulfamethoxazole-Trimethoprim 80mg-16 mg/ml solution ESI Lederle Generics
Penicillin V Potassium V potassium 500mg tablet UDL Laboratories Inc

Allergies
Lactose
Shellfish


Reason for Visit
Shortness of breath following pneumonia.

Narrative History
Patient diagnosed with Asthma 01/03. Has severe shortness of breath, fainting spells 2 week after pneumonia diagnosis.

Review of Systems
Shortness of breath. Coughing. Swimming in the head. Chest pain.

Vital Signs
BLOOD PRESSURE   119/61
PULSE   125.7
TEMPERATURE   97.8 F
WEIGHT   114.4 lb
O2 SAT   81


Exam
Pharynx normal.? PERRL, EOM full.
Neck without nodes.
Chest is clear to P&A.
Cor: normal S1, S2, without gallop or murmur.


External Labs
CXR reviewed: clear.

Impression
Generally doing well.? Low 02 sat. Pain reasonably well controlled.? No signs of active infection.

Plan
1.)? Nebulizer
2.)? RTC 2 weeks.
```



In [ ]:
# !pip install "transformers[torch]" "huggingface_hub[inference]"
!pip install llama-index

In [ ]:
%env HF_TOKEN=HUGGINGFACE_TOKEN

env: HF_TOKEN=HUGGINGFACE_TOKEN


In [ ]:
import os
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index.llms import HuggingFaceInferenceAPI

llm = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha", token=os.environ['HF_TOKEN'])
service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm, embed_model="local")

docs = SimpleDirectoryReader("/content/patient_data").load_data()
index = VectorStoreIndex.from_documents(docs, service_context=service_context)
query_engine = index.as_query_engine()

user = "User:"
gpdoc = "GPDoc:"
print("*** Patient History ***")
question = "Can you provide more details about the patient's medical history, including past illnesses or surgeries?"
answer = query_engine.query(question)
print(user, question)
print(gpdoc, answer.response.strip())
print()

print("*** Follow-ups ***")
question = "Could you clarify the follow-up recommendations mentioned in the notes?"
answer = query_engine.query(question)
print(user, question)
print(gpdoc, answer.response.strip())
print()

print("*** Progress Update ***")
question = "Are there any new symptoms or changes in the patient's health status that we should be aware of?"
answer = query_engine.query(question)
print(user, question)
print(gpdoc, answer.response.strip())
print()

print("*** Medication Adjustments ***")
question = "What are the current dosages and frequencies of the medications prescribed?"
answer = query_engine.query(question)
print(user, question)
print(gpdoc, answer.response.strip())
print()

print("*** Treatment ***")
question = "What is the rationale behind starting Singulair for this patient?"
answer = query_engine.query(question)
print(user, question)
print(gpdoc, answer.response.strip())
print()

print("*** Diagnosis ***")
question = "Can you provide more details about the symptoms or findings that led to this diagnosis?"
answer = query_engine.query(question)
print(user, question)
print(gpdoc, answer.response.strip())
print()

print("*** Procedures ***")
question = "Can you describe any complications or post-procedure care instructions associated with surgeries?"
answer = query_engine.query(question)
print(user, question)
print(gpdoc, answer.response.strip())

*** Patient History ***
User: Can you provide more details about the patient's medical history, including past illnesses or surgeries?
GPDoc: The patient's medical history includes asthma, salt-rheum, and back pain. However, there is no mention of any surgeries in the given context information.

*** Follow-ups ***
User: Could you clarify the follow-up recommendations mentioned in the notes?
GPDoc: In the first note, the patient was admitted to the ER for dehydration and nausea, vomiting, and diarrhea. The follow-up recommendations would likely involve close monitoring of the patient's hydration status and management of any ongoing gastrointestinal symptoms. The patient may need to continue with IV fluids and antibiotics, and may require further testing to determine the underlying cause of the gastroenteritis.

In the second note, the patient is suspected to be pregnant based on a positive home pregnancy test. The follow-up recommendations would likely involve confirming the pregnancy w